# This notebook demonstrates how we fine-tuned a GPT-2 model on a Vietnamese poetry dataset using [Kaggle Notebooks](https://www.kaggle.com/code). 

In [ ]:
import os

WORKSPACE = "/kaggle/working/vietnamese-poems/Vietnamese_Poatry_Model/"
os.makedirs(WORKSPACE, exist_ok=True)

In [4]:
import mlflow
mlflow.set_tracking_uri(f"file://{WORKSPACE}/mlruns")
mlflow.set_experiment("gpt2-viwiki")

2025/07/24 15:11:12 INFO mlflow.tracking.fluent: Experiment with name 'gpt2-viwiki' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///kaggle/working/vietnamese-poems/Vietnamese_Poetry_Model//mlruns/953792292206460793', creation_time=1753369872914, experiment_id='953792292206460793', last_update_time=1753369872914, lifecycle_stage='active', name='gpt2-viwiki', tags={}>

In [ ]:
import pandas as pd

from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import warnings
warnings.filterwarnings("ignore")


2025-07-24 15:11:27.713846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753369887.914246      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753369887.971187      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
DATA_PATH = "/kaggle/input/vietnamese-poem2/poem_dataset.csv"
df = pd.read_csv(DATA_PATH)
df = df[df["content"].notna()]
df = df[df["content"].apply(lambda x: x is not None)]
df = df[df["content"].apply(lambda x: str(x).strip() != "")]
df

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
2,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
3,3,"“Đến, nhiều nơi để đến”","Đến, nhiều nơi để đến\nVề, trở lại với mình\nC...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Đừng bao giờ dại dột”,Đừng bao giờ dại dột\nĐem chuyện riêng của mìn...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...
185,95,Ám ảnh sông xưa,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
186,96,Áng dương không biết sầu,Áng dương không biết sầu\nNằm mãi ở trên cao\n...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
187,97,Anh,Cây bút gẫy trong tay\nCặn mực khô đáy lọ\nÁnh...,19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...
188,98,Anh biết,Không có anh để già\nLàm sao em được trẻ\nMuốn...,NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [7]:
def split_content(content):
    samples = []
    poem_parts = content.split("\n\n")
    for poem_part in poem_parts:
        poem_in_lines = poem_part.split("\n")
        if len(poem_in_lines) == 4:
            samples.append(poem_in_lines)
    return samples

df["content"] = df["content"].apply(lambda x:split_content(x))
df_exploded = df.explode("content")
df_exploded.reset_index(drop=True, inplace=True)
df_exploded = df_exploded.dropna(subset=["content"])
df_exploded


,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,"[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhiề...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,0,“Cái làm ta hạnh phúc”,"[Rồi thêm chút công việc, Cho ta làm hàng ngày...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,1,“Chiều vừa xốp trên tay”,"[Chiều vừa xốp trên tay, Chợt nghe thoáng ong ...","Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,1,“Chiều vừa xốp trên tay”,"[Ớt đỏ sao cứ đỏ, Táo chín cho thật vàng, Em đ...","Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
4,2,“Dưới giàn hoa thiên lý...”,"[Dưới giàn hoa thiên lý, Một mình anh đang ngồ...","Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
...,...,...,...,...,...
481,98,Anh biết,"[Đất hiến dâng tuổi trẻ, Trời buông thả tuổi g...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
482,99,Anh bốn mùa lập đông,"[Buồn, thì buồn thiệt đó, Em à, có biết không,...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
483,99,Anh bốn mùa lập đông,"[Vâng, phiến buồn còn đó, Em nào biết phải khô...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
484,99,Anh bốn mùa lập đông,"[Ừ, nỗi buồn vậy đó, Em đừng vội bâng khuâng, ...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [8]:
df_exploded["content"] = df_exploded["content"].apply(lambda x: "\n".join(x))
df_exploded = df_exploded[df_exploded["content"].notnull()]
df_exploded = df_exploded[df_exploded["content"].apply(lambda x: isinstance(x, str))]
df_exploded

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,0,“Cái làm ta hạnh phúc”,Rồi thêm chút công việc\nCho ta làm hàng ngày\...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,1,“Chiều vừa xốp trên tay”,Ớt đỏ sao cứ đỏ\nTáo chín cho thật vàng\nEm đẹ...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
4,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
...,...,...,...,...,...
481,98,Anh biết,Đất hiến dâng tuổi trẻ\nTrời buông thả tuổi gi...,NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
482,99,Anh bốn mùa lập đông,"Buồn, thì buồn thiệt đó\nEm à, có biết không\n...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
483,99,Anh bốn mùa lập đông,"Vâng, phiến buồn còn đó\nEm nào biết phải khôn...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
484,99,Anh bốn mùa lập đông,"Ừ, nỗi buồn vậy đó\nEm đừng vội bâng khuâng\nV...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [9]:
TEST_SIZE = 0.1
raw_dataset = Dataset.from_pandas(df_exploded)
poem_dataset = raw_dataset.train_test_split(test_size=TEST_SIZE)
poem_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'content', 'source', 'url', '__index_level_0__'],
        num_rows: 396
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'content', 'source', 'url', '__index_level_0__'],
        num_rows: 45
    })
})

In [ ]:
model = GPT2LMHeadModel.from_pretrained("danghuy1999/gpt2-viwiki")
tokenizer = GPT2Tokenizer.from_pretrained("danghuy1999/gpt2-viwiki")

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
MAX_SEQ_LEN = 100

def preprocess_function(batch):
    return tokenizer(
        batch["content"],
        max_length=MAX_SEQ_LEN,
        truncation=True,
        padding="max_length"
    )

tokenized_poem_dataset = poem_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=poem_dataset["train"].column_names
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


In [ ]:
training_args = TrainingArguments(
    output_dir=f"{WORKSPACE}/gpt2-poetry", 
    save_strategy="epoch",
    eval_strategy="epoch",
    save_total_limit=1,
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    fp16=True,
    report_to="mlflow", 
    run_name="gpt2-poem-run",
    logging_dir="/kaggle/working/logs", 
    logging_steps=10
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_poem_dataset["train"],
    eval_dataset=tokenized_poem_dataset["test"],
    data_collator=data_collator
)

trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.449800,6.730793
2,3.235200,6.482757
3,3.126300,6.318743
4,3.009300,6.234958
5,2.953500,6.155170
6,2.920100,6.095672
7,2.892500,6.050497
8,2.923900,6.011653
9,2.826200,5.993170
10,2.842400,5.988188


TrainOutput(global_step=250, training_loss=3.040596206665039, metrics={'train_runtime': 101.435, 'train_samples_per_second': 39.04, 'train_steps_per_second': 2.465, 'total_flos': 202093056000000.0, 'train_loss': 3.040596206665039, 'epoch': 10.0})

In [14]:
def generate_poem(
    prompt,
    max_length=50,
    temperature=0.8,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2
):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    input_ids = inputs["input_ids"].to(model.device)

    punctuation_list = [
        ".", ",", "!", "?", ";", ":", "\"", "'", "“", "”", "‘", "’", "(", ")", "[", "]", "{", "}", "~", "`", "-", "_", "*", "/", "\\", "|", "<", ">", "=", "+"
    ]
    
    bad_words = [
        tokenizer.encode(p, add_special_tokens=False) for p in punctuation_list
    ]
    
    bad_words_ids = [bw for bw in bad_words if len(bw) > 0]

    output_ids = model.generate(
        input_ids=input_ids,
        do_sample=True,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        pad_token_id=tokenizer.eos_token_id,
        bad_words_ids=bad_words_ids
    )

    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text


In [15]:
import re
import unicodedata

def normalize_word(word):
    word = unicodedata.normalize('NFD', word)
    word = re.sub(r'[\u0300-\u036f]', '', word) 
    return word.lower()

def is_rhyme(word1, word2):
    word1 = normalize_word(word1)
    word2 = normalize_word(word2)
    return word1[-2:] == word2[-2:]

def evaluate_poem_rules(poem_text, bad_words=None):
    if bad_words is None:
        bad_words = []

    lines = [line.strip() for line in poem_text.strip().split('\n') if line.strip()]
    total_lines = len(lines)
    valid_5_words = 0
    badword_count = 0
    rhyme_score = 0
    rhyme_pairs = 0

    for i, line in enumerate(lines):
        words = line.split()
        
        if len(words) == 5:
            valid_5_words += 1

        if i + 2 < total_lines:
            last1 = lines[i].split()[-1]
            last2 = lines[i+2].split()[-1]
            if is_rhyme(last1, last2):
                rhyme_score += 1
            rhyme_pairs += 1

    return {
        "total_lines": total_lines,
        "percent_5_words_line": valid_5_words / total_lines if total_lines else 0,
        "bad_word_count": badword_count,
        "rhyme_rate": rhyme_score / rhyme_pairs if rhyme_pairs else 0,
    }


In [17]:
seeds = [
    "Trăng vàng treo đầu núi",
    "Gió thổi qua cành khô",
    "Lá rơi trên mặt đất",
    "Em cười như nắng mai",
    "Sông trôi về nơi xa",
    "Trời xanh không một áng",
    "Hoa nở trong chiều mưa",
    "Tình anh như sóng biển",
    "Bước chân qua phố vắng",
    "Học, học nữa, học mãi",
    "Chiều buông nhẹ bên sông",
    "Lặng lẽ hàng cây đứng",
    "Giọt sương đọng đầu cành",
    "Tiếng ve ngân cuối hạ",
    "Ánh hoàng hôn phai dần",
    "Trà nguội trong đêm vắng",
    "Ngọn đèn khuya le lói",
    "Bàn chân qua ngõ nhỏ",
    "Mẹ ru bằng khói bếp",
    "Nhớ thương về một thuở"
]

for i, seed in enumerate(seeds):
    print(f"\nBài thơ {i+1}: \"{seed}\"")
    poem = generate_poem(prompt=seed + "\n")
    print(poem)

    eval_result = evaluate_poem_rules(poem)
    print("📊 Đánh giá:")
    print(f"   - Số dòng: {eval_result['total_lines']}")
    print(f"   - % dòng đúng 5 chữ: {eval_result['percent_5_words_line'] * 100:.1f}%")
    print(f"   - Tỷ lệ vần đúng: {eval_result['rhyme_rate'] * 100:.1f}%")


Bài thơ 1: "Trăng vàng treo đầu núi"
Trăng vàng treo đầu núi
Mà dốc dốc núi dài
Như vậy ở đây không có gì
Nhưng là con người sống lại
Chứt cái mình không thấy
Khi một con người chết thì chưa biết
Khây xa bờ biển

📊 Đánh giá:
   - Số dòng: 7
   - % dòng đúng 5 chữ: 42.9%
   - Tỷ lệ vần đúng: 20.0%

Bài thơ 2: "Gió thổi qua cành khô"
Gió thổi qua cành khô
Không có nước trong rừng
Luyện vui hơn trời mưa
Một con đường đi trời mưa
Nên ngày càng nhiều hơn nữa
Vua cầu mùa đông hơn nữa
Để bay trở lại trời mưa
Điêu khó
📊 Đánh giá:
   - Số dòng: 8
   - % dòng đúng 5 chữ: 37.5%
   - Tỷ lệ vần đúng: 50.0%

Bài thơ 3: "Lá rơi trên mặt đất"
Lá rơi trên mặt đất
Khi gặp một con người
Cả lòng mình thoát xuống
Đã thành mưa khắp nơi
Nên có người con và mẹ
Không cần ai nào không phải
Cho đến thời gian giữa hai con ấy
Nó mang lại
📊 Đánh giá:
   - Số dòng: 8
   - % dòng đúng 5 chữ: 50.0%
   - Tỷ lệ vần đúng: 33.3%

Bài thơ 4: "Em cười như nắng mai"
Em cười như nắng mai
Một người con chim linh hồn
Đôi khi đ